### 🎯 What will you learn?
- How to detect harmful algal blooms (HABs) in coastal and brackish lakes using Sentinel-2 data.

- How to use multiple spectral indices related to water, chlorophyll, and floating algae.

- How to apply a Random Forest (RF) model for binary classification (Bloom / No Bloom).

- How to evaluate model performance


---

## Why Random Forest?
Random Forest (RF) is an ensemble machine learning algorithm that combines multiple decision trees to improve classification accuracy and robustness.  

In remote sensing applications, RF is particularly powerful because it:
- Handles non-linear relationships between spectral bands and indices  

- Is resistant to overfitting compared to single-tree models

- Works well with mixed inputs (bands + spectral indices)  

- Does not require predefined thresholds, as class separation is learned directly from the data  

---

## Spectral Indices Used


| Index | Description | Typical Range | Equation |
|-------|-------------|---------------|----------|
| MNDWI | Modified Normalized Difference Water Index – highlights water bodies | -1 to 1 | (Green - SWIR1) / (Green + SWIR1) |
| NDCI  | Normalized Difference Chlorophyll Index – sensitive to chlorophyll-a | -1 to 1 | (RedEdge - Red) / (RedEdge + Red) |
| AFAI  | Alternative Floating Algae Index – detects floating algae | Variable | NIR - (Red + (SWIR - Red) * ((λ_NIR - λ_Red) / (λ_SWIR - λ_Red))) |
| MCI   | Maximum Chlorophyll Index – high chlorophyll concentration | Variable | RE2 - RE1 - (RE3 - RE1) * ((λ_RE2 - λ_RE1)/(λ_RE3 - λ_RE1)) |
| ABDI  | Algal Bloom Difference Index – distinguishes bloom from turbid water | -1 to 1 | (Blue - Red) / (Blue + Red) |



Note : This is a binary classification problem where HAB presence is approximated using spectral and index-based features

## Workflow Overview

1. Define the Area of Interest (Burullus Lake).

2. Load and preprocess Sentinel-2 imagery (cloud masking and compositing).

3. Compute selected spectral indices.

4. Visualize water and bloom-related indices for interpretation.

5. Create training samples and split them into training and testing sets.

6. Train a Random Forest classifier.

7. Evaluate model accuracy using multiple metrics.

8. Google Drive Export


In [ ]:
import ee
import geemap



In [ ]:
# Authenticate to Earth Engine.
ee.Authenticate()

# Initialize Earth Engine with your project ID
ee.Initialize(project =  'ee-nouribrahim25')

**DEFINE AREA OF INTEREST (AOI) Burullus Lake**

In [ ]:
coors =[
    [[31.118924, 31.556277],
    [30.889509, 31.385269],
    [30.573548, 31.379407],
    [30.557063, 31.447384],
    [31.00353, 31.591377],
    [31.118924, 31.556277]]
]

aoi = ee.Geometry.Polygon(coors)
aoi

In [ ]:
# show AOI on the map
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.add_layer(aoi , {} , 'AOI')
Map.centerObject(aoi , 10)
Map

**Load Sentinel-2 & Cloud Mask**


In [ ]:
# Date Range
start_date = '2025-05-01'
end_date = '2025-09-01'

# define cloud mask function
def cloud_mask (image):
    qa = image.select('QA60')
    cloud = qa.bitwiseAnd(1 << 10).eq(0)
    cirrus = qa.bitwiseAnd(1 << 11).eq(0)
    return image.updateMask(cloud.And(cirrus)).divide(10000)


# Load Sentinel-2 data
s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterBounds(aoi)
      .filterDate(start_date , end_date)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
      .map(cloud_mask)
)
s2.size()

**Manual Spectral Indices Functions**


In [ ]:
def add_indices(img):

    # MNDWI
    mndwi = img.expression(
        '(G - S) / (G + S)', {
            'G': img.select('B3'),
            'S': img.select('B11')
        }
    ).rename('MNDWI')

    # NDCI
    ndci = img.expression(
        '(RE - R) / (RE + R)', {
            'RE': img.select('B5'),
            'R': img.select('B4')
        }
    ).rename('NDCI')

    # AFAI
    afai = img.expression(
        'B8 - (B4 + (B11 - B4) * ((842 - 665) / (1610 - 665)))',
        {
            'B8': img.select('B8'),
            'B4': img.select('B4'),
            'B11': img.select('B11')
        }
    ).rename('AFAI')

    # MCI
    mci = img.expression(
        'B6 - B5 - (B7 - B5) * ((740 - 705) / (783 - 705))',
        {
            'B5': img.select('B5'),
            'B6': img.select('B6'),
            'B7': img.select('B7')
        }
    ).rename('MCI')

    # ABDI
    abdi = img.expression(
        '(B - R) / (B + R)', {
            'B': img.select('B2'),
            'R': img.select('B4')
        }
    ).rename('ABDI')

    return img.addBands([mndwi, ndci, afai, mci, abdi])


**Apply Indices and make median composite**

In [ ]:
s2_indices = s2.map(add_indices)

composite = s2_indices.median().clip(aoi)
composite

**Apply Water Mask**

In [ ]:
water_mask = composite.select('MNDWI').gt(0)   # Water is usually > 0 in MNDWI
image = composite.updateMask(water_mask)


**Visualization (False Color and NDCI)**

In [ ]:
# Visualization False Color Composite
false_color_vis = {
    'bands': ['B8', 'B4', 'B3'],  # NIR, Red, Green
    'min': 0.02,
    'max': 0.4,
}

ndci_vis = {
    'min': -0.2,
    'max': 0.4,
    'palette': ['blue', 'cyan', 'yellow', 'red']
}

Map.add_layer(image, false_color_vis, 'False Color', False)
Map.add_layer(image.select('NDCI'), ndci_vis, 'NDCI', False)
Map


##**MACHINE LEARNING**

**Create labels based on the NDCI index**

Using 0.15 as the threshold for NDCI ,

1 = Bloom (Danger), 0 = No Bloom

**NOTE:** The labels are derived from NDCI thresholding and represent proxy bloom conditions, not field measurements.

In [ ]:
# Features List (Bands)
bands = ['MNDWI','AFAI','MCI','ABDI','B5', 'B8']    #B5 (Red Edge - 705nm) and B8 (NIR - 842nm)

# Create the Label
label = image.select('NDCI').gt(0.15).rename('label')

# Create the Training Stack (Features + Label)
stack = image.select(bands).addBands(label)

stack

**Stratified Sampling**

In [ ]:
# Sample Size
sample_size = 1500   #per class

# Sampling
samples = stack.stratifiedSample(
    numPoints= sample_size,
    classBand='label',
    region=aoi,
    scale=20,
    seed= 42,
    geometries=True,
    tileScale=16      # Helps avoid 'Memory Limit Exceeded'
)

# show samples on the map , show size and show the first sample point
Map.addLayer(samples, {}, 'samples', False)
print(samples.size().getInfo())
print(samples.first().getInfo())
Map

**Train/Test Split (70 / 30)**

In [ ]:
# Add random column for splitting
samples = samples.randomColumn("random")

# Train Test Split (70% - 30%)
training = samples.filter(ee.Filter.lt("random", 0.7))
test  = samples.filter(ee.Filter.gte("random", 0.7))


In [ ]:
print(training.size().getInfo())
print(test.size().getInfo())

**Train Random Forest (100 Trees)**


In [ ]:
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training,
    classProperty='label',
    inputProperties=bands
)


In [ ]:
# Classify the image
classified_image = image.select(bands).classify(classifier)

# Display Algal Bloom Danger Map
class_vis = {'min':0 , 'max':1 , 'palette': ['#1f4e79', '#d62728']}
legend_dict = {'No Bloom': '#1f4e79','Bloom': '#d62728'}

Map.remove_legend()

Map.add_layer(classified_image, class_vis, 'Classified Image')
Map.add_legend(title='Harmful Algal Bloom', legend_dict=legend_dict)

Map

**Accuracy Assessment**

In [ ]:
# Train Accuracy

# Confusion Matrix
train_accuracy = classifier.confusionMatrix()

print("\n=== Training Accuracy ===")
train_accuracy_list = train_accuracy.getInfo()
# Pretty print as a table
for row in train_accuracy_list:
    print(row)

# Accuracy metrics
print("\nOverall Accuracy:", train_accuracy.accuracy().getInfo())
print("Kappa:", train_accuracy.kappa().getInfo())
print("Producer Accuracy (Recall):", train_accuracy.producersAccuracy().getInfo())
print('Consumer\'s Accuracy (Precision):', train_accuracy.consumersAccuracy().getInfo())
print('F1 Scores:', train_accuracy.fscore().getInfo())

In [ ]:
# Test Accuracy

# Classify the test set to add the 'classification' property
classified_test = test.classify(classifier)

# Confusion Matrix
validation_accuracy = classified_test.errorMatrix('label', 'classification')

print("\n=== Confusion Matrix ===")
validation_accuracy_list = validation_accuracy.getInfo()

# Pretty print as a table
for row in validation_accuracy_list:
    print(row)

# Accuracy metrics
print("\nOverall Accuracy:", validation_accuracy.accuracy().getInfo())
print("Kappa:", validation_accuracy.kappa().getInfo())
print("Producer Accuracy (Recall):", validation_accuracy.producersAccuracy().getInfo())
print('Consumer\'s Accuracy (Precision):', validation_accuracy.consumersAccuracy().getInfo())
print('F1 Scores:', validation_accuracy.fscore().getInfo())

**Feature Importance**

In [ ]:
classifier.explain()


**HAP Area Percentage**

In [ ]:
pixel_area = ee.Image.pixelArea()

hab_area = classified_image.eq(1).multiply(pixel_area).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=aoi,
    scale=100,  # Increased scale to reduce memory usage further
    maxPixels=1e13
).get('classification')

water_area = water_mask.multiply(pixel_area).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=aoi,
    scale=100,  # Increased scale to reduce memory usage further
    maxPixels=1e13
).get('MNDWI')

hab_percentage = ee.Number(hab_area).divide(water_area).multiply(100)

hab_percentage.getInfo()

**EXPORT THE CLASSIFIED IMAGE TO GOOGLE DRIVE**

In [ ]:
export_classified = ee.batch.Export.image.toDrive(
    image= classified_image.uint8(),      # convert to unsigned 8-bit
    description='Burullus_Bloom_Classification_RF',
    folder='GEE_Exports',          # Your folder in Google Drive
    fileNamePrefix='HAB_RF',
    scale=20,
    region=aoi,
    maxPixels=1e13
)

export_classified.start()